# Microsoft 365 Defender (M365D) (previously MTP) REST API FUN

- DOCUMENTATION on the list of exposed M365D/MTP APIs: [here!](https://docs.microsoft.com/en-us/microsoft-365/security/mtp/api-supported?view=o365-worldwide)


|#|Topic|Description|Link|
|:-|:-|:-|:-|
|1|Advanced Hunting|Run queries from API.|[link](https://docs.microsoft.com/en-us/microsoft-365/security/mtp/api-advanced-hunting?view=o365-worldwide)|
|2|Incident |List and update incidents, along with other practical tasks.|[link](https://docs.microsoft.com/en-us/microsoft-365/security/mtp/api-incident?view=o365-worldwide)|


## Requirements for accessing a MDE via REST API

To access the MDE of someone (ex. a customer) you need to:
1. Create an App in the Azure AD. An easy documentation [HERE!](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/exposed-apis-create-app-partners)
After setting-up the App, check in the overview the following info: (1) Application (client) ID, (2) Directory (tenant) ID, and (3) Secret value

3. VERY IMPORTANT. The global admin should give you consent to access the App (client_id): $https://login.microsoftonline.com/common/oauth2/authorize?prompt=consent&client_id=<HERE-THE-CLIENT-ID>&response_type=code&sso_reload=true$

## Python libraries used

In [ ]:
import pandas as pd
import json
import urllib.parse

<h1 align='center'>=====================================<br>VERY IMPORTANT<br>ALL FUNCTIONS DEPENDS ON THIS ACCESS TOKEN!!</h1>

## FUNCTION mde_getaadtoken()

In [ ]:
def mde_getaadtoken(tenantId, appId, appSecret):
    url = "https://login.windows.net/%s/oauth2/token" % (tenantId)
    resourceAppIdUri = 'https://api.security.microsoft.com'
    body = {
        'resource' : resourceAppIdUri,
        'client_id' : appId,
        'client_secret' : appSecret,
        'grant_type' : 'client_credentials'
    }
    data = urllib.parse.urlencode(body).encode("utf-8")
    req = urllib.request.Request(url, data)
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    aadToken = jsonResponse["access_token"]
    return aadToken

In [ ]:
access_token = mde_getaadtoken(tenantId, appId, appSecret)

<h1 align='center'>=====================================<br>ADVANCED HUNTING API<br>https://docs.microsoft.com/en-us/microsoft-365/security/mtp/api-advanced-hunting?view=o365-worldwide</h1>

List of queries: https://github.com/microsoft/Microsoft-365-Defender-Hunting-Queries

In [ ]:
def m365d_advancedquery_run(access_token,kql_query):
    url = "https://api.security.microsoft.com/api/advancedhunting/run"
    req = urllib.request.Request(url, 
                                 headers={'Authorization' : "Bearer " + access_token, 'Content-Type':'application/json'},
                                 data=bytes(json.dumps({"Query": kql_query}), encoding="utf-8"))
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['Results'])

In [ ]:
kql_query = '''AlertInfo|where Timestamp > ago (30d)'''

In [ ]:
m365d_advancedquery_run(access_token,kql_query)

<h1 align='center'>=====================================<br>INCIDENT API<br>https://docs.microsoft.com/en-us/microsoft-365/security/mtp/api-incident?view=o365-worldwide</h1>

|Method	| Return Type |Description  |
|:- |:-|:-|
|List incidents|	Incident list|	Get a list of incidents.|
|Update incident|	Incident|	Update a specific incident.|

## METHOD List incidents

In [ ]:
def m365d_incidents_listincidents(access_token):
    url = "https://api.security.microsoft.com/api/incidents"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

## METHOD Update incident

In [ ]:
def m365d_incidents_updateincidents(access_token,incident_id):
    url = "https://api.security.microsoft.com/api/incidents/"+str(incident_id)
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return jsonResponse